In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 
                 'C:\\Users\\troy\\Documents\\Excel Sheets', 'C:/Users/troy/Documents/Code/Libraries'])

from datetime import date, datetime
import os
import numpy as np
import pandas as pd

trade_dump_folder = r"C:\Users\troy\Documents\Excel Sheets\Blotter Dump"

class Portfolio:

    option_fields = ['']

    def __init__(self, data_date):
        self.data_date = data_date

        # determine the file to be read and
        self.expected_filename = "Dump_%s.csv" % today.strftime("%Y%m%d")
        self.expected_file_path = os.path.join(trade_dump_folder, self.expected_filename)
        self.raw_data = self.read_csv()
        self.trader_df = None

    def read_csv(self):
        parse_dates = ['Trade Date', 'Settlement Date', 'Maturity Date', 'AutoClose Date',
                       'Premium Date', 'Barrier1 Effective Date', 'Barrier1 Maturity Date',
                       'Barrier2 Effective Date', 'Barrier2 Maturity Date', 'First Notice Date',
                       'Upcoming Fixing Date', 'Repo Close Date']
        notional_fields = ['Notional', 'Secondary Notional', 'Currency1 Notional', 'Currency2 Notional',
                           'Trade Price', 'Accrued Interest']
        dtypes = {'Block Id': 'Int64',
                  'MarketBlockId': 'Int64',
                  'Asset Type': 'str',
                  'TradeID': 'Int64',
                  'Trade Name': 'str',
                  'Security Id': 'str',
                  'Account Name': 'str',
                  'Trader Name': 'str',
                  'Theme Name': 'str',
                  'Asset Class': 'str',
                  'Entered By User Name': 'str',
                  'Approved By User Name': 'str',
                  'Verified By User Name': 'str',
                  'Tradar Id': 'Int64',
                  'Trade Date': 'str',
                  'Settlement Date': 'str',
                  'Upcoming Fixing Date': 'str',
                  'Repo Close Date': 'str',
                  'Maturity Date': 'str',
                  'Is Trade Open': 'str',
                  'Auto Open Close': 'str',
                  'ISIN': 'str',
                  'BBG Type': 'str',
                  'Description': 'str',
                  'Currency Code': 'str',
                  'Notional': 'str',
                  'Secondary CCY': 'str',
                  'Secondary Notional': 'str',
                  'Trade Type': 'str',
                  'Trade Price': 'str',
                  'Rate': 'float',
                  'Counter Party': 'str',
                  'Trade BBG Type': 'str',
                  'Country Name': 'str',
                  'Capital Allocation': 'float',
                  'Trade Name Capital Perc': 'float',
                  'Number Of Contracts': 'float',
                  'Bond Issuer': 'str',
                  'Tradar Pay Account': 'str',
                  'Tradar Account': 'str',
                  'Op Notes': 'str',
                  'AutoClose Date': 'str',
                  'FXSwap Package': 'str',
                  'Barrier1': 'float',
                  'Barrier1 Effective Date': 'str',
                  'Barrier1 Maturity Date': 'str',
                  'Barrier2': 'float',
                  'Barrier2 Effective Date': 'str',
                  'Barrier2 Maturity Date': 'str',
                  'Strike2': 'float',
                  'CuttOfTime2': 'str',
                  'Receive Ccy2': 'str',
                  'Pay Ccy2': 'str',
                  'Option PayOut2': 'str',
                  'Premium': 'float',
                  'Upfront': 'float',
                  'CutOffTime': 'str',
                  'Option Type': 'str',
                  'Option Style': 'str',
                  'Option PayOut': 'str',
                  'Currency 1': 'str',
                  'Currency 2': 'str',
                  'Currency1 Notional': 'str',
                  'Currency2 Notional': 'str',
                  'Upfront Ccy': 'str',
                  'Premium Date': 'str',
                  'Accrued Interest': 'str',
                  'Fixing Trade Id': 'Int64',
                  'Benchmark 1': 'str',
                  'Benchmark 2': 'str',
                  'Is Matured': 'bool',
                  'Is NDIRS': 'bool',
                  'Repo Settlement Amount': 'float',
                  'First Notice Date': 'str'
                  }
        df = pd.read_csv(self.expected_file_path, encoding='latin1', thousands=',',
                         skipinitialspace=True, parse_dates=parse_dates, dtype=dtypes)
        for fld in notional_fields:
            df[fld] = [float(n.strip().replace(',', '').replace('(', '-').replace(')', ''))
                       if isinstance(n, str) else n for n in df[fld]]

        df['#_or_notional_mio'] = np.where((df['Asset Type'] == 'Futures') | (df['Asset Type'] == 'Futures Options'),
                                           df['Number Of Contracts'], df['Notional'] / 1e6)
        print("data loaded ...")
        return df

    @staticmethod
    def __construct_filter_string__( df ,col, val, cond):
        if cond=='EQ':
            return df[col] == val
        elif cond=='NE':
            return df[col] != val
        elif cond == 'LE':
            return df[col] <= val
        elif cond == 'LT':
            return df[col] < val
        elif cond == 'GE':
            return df[col] >= val
        elif cond == 'GT':
            return df[col] > val
        elif cond == 'ISIN':
            return df[col].isin(val)
        elif cond == 'STR.CONTAINS':
            return df[col].str.contains(val)
        else:
            return None

    def _apply_equality_filters(self, df, cols, values, conditional, aggregator='or'):
        assert len(cols)==len(values)
        key_values = zip(cols[1:], values[1:])
        filter = self.__construct_filter_string__(df , cols[0], values[0], conditional[0])
        for k, v in key_values:
            if aggregator.upper() == 'OR':
                filter = filter |  df[k] == v
            elif aggregator.upper() == 'AND':
                filter = filter & df[k] == v
            else:
                raise ValueError('aggregator should be in [OR, AND]')

    # standard trades

    def apply_trader_filters(self, trader_name, remove_closed_trades=True):
        trader_df = self.raw_data[self.raw_data["Trader Name"] == trader_name]
        if remove_closed_trades:
            trader_df = self.trader_df[self.trader_df['Is Trade Open'] == 'TRUE']
        return trader_df

    # Some standard filters

    @staticmethod
    def get_trades_by_trader(df, trader_name):
        result = df[df['Trader Name'] == trader_name]
        return result

    @staticmethod
    def get_option_trades(df):
        option_df = df[
            (df['Asset Type'] == 'Futures Options') | (df['Asset Type'] == 'Swaption') | (
                        df['Asset Type'] == 'Currency Options')]
        return option_df

    @staticmethod
    def filter_asset_class(df, asset_class):
        asset_classes = {'EQUITY':'Equity', 'FX': 'FX', 'RATES': 'Rates', 'CREDIT': 'Credit'}
        if asset_class.upper() not in asset_classes:
            raise ValueError('%s is invalid - valid values are %s' % (asset_class, list(asset_classes.values())))
        if isinstance(asset_class, str):
            return df[df['Asset Class'] == asset_classes[asset_class.upper()]]
        else:
            if not isinstance(asset_class, collections.Iterable):
                raise ValueError('asset class should either be a string or a list of strings')
            else:
                assets = [asset_classes[a.upper()] for a in asset_classes]
                return df[df['Asset Class'].isin(assets)]

    @staticmethod
    def filter_asset_type(df, asset_type):
        possible_asset_types = Portfolio.asset_types(df)
        asset_types = {v.upper():v for v in possible_asset_types}
        if isinstance(asset_type, str):
            return df[df['Asset Type']==asset_types[asset_type.upper()]]
        else:
            if not isinstance(asset_type, collections.Iterable):
                raise ValueError('asset_type should either be a string or a list of strings')
            else:
                assets = [asset_types[a.upper()] for a in asset_types]
                return df[df['Asset Type'].isin(assets)]

    @staticmethod
    def filter_by_block_id(df, block_id):
        if isinstance(block_id, int):
            return df[df['Block Id']==block_id]
        else:
            if isinstance(block_id, collections.Iterable):
                return df[df['Block Id'].isin(block_id)]
            else:
                raise ValueError('block_id should be an integer or a list of integers')

    @staticmethod
    def filter_by_trade_date(df, trade_date, cond='EQ'):
        filter = Portfolio.__construct_filter_string__(df, 'Trade Date', trade_date, cond)
        if filter:
            return df[filter]

    @staticmethod
    def filter_by_maturity_date(df, maturity_date, cond='EQ'):
        filter = Portfolio.__construct_filter_string__(df, 'Maturity Date', maturity_date, cond)
        if filter:
            return df[filter]

    @staticmethod
    def filter_only_open_trades(df):
        return df[df['Is Trade Open']=='TRUE']

    @staticmethod
    def filter_only_closed_trades(df):
        return df[df['Is Trade Open']=='FALSE']

    @staticmethod
    def create_position_pivot(df):
        accounts = [v for v in np.unique(df['Account Name']) if v != 'MassPRIM-B']

        # Create the Pivot
        trades = pd.pivot_table(data=df, values='#_or_notional_mio',
                                index=['Trade Name', 'Security Id'], columns='Account Name', margins=True,
                                margins_name='Total', aggfunc='sum')
        trade_df = trades.reset_index()
        if 'MassPRIM-B' in trade_df.columns:
            trade_df = trade_df.drop(['MassPRIM-B'], axis=1)
        trade_df = trade_df.round(2)

        # Remove isins with 0 values
        zero_isins = (trade_df[accounts[0]] == 0) | np.isnan(trade_df[accounts[0]])
        for a in accounts:
            zero_isins = zero_isins & ((trade_df[a] == 0) | (np.isnan(trade_df[a])))
        trade_df = trade_df[~zero_isins]

        # Remove duplicated Trade names for better display
        trade_df['Trade Name'] = np.where(trade_df['Trade Name'].duplicated(), '', trade_df['Trade Name'])

        return trade_df

    # Field infos
    @staticmethod
    def trade_names(df):
        return np.unique(df['Trade Name'])

    @staticmethod
    def open_trades(df):
        return np.unique(df[df['Is Trade Open'] == 'TRUE']['Trade Name'])

    @staticmethod
    def closed_trades(df):
        return np.unique(df[df['Is Trade Open'] == 'FALSE']['Trade Name'])

    @ staticmethod
    def asset_types(df):
        return np.unique(df['Asset Type'])

    @staticmethod
    def asset_classes(df):
        return np.unique(df['Asset Class'])

    @staticmethod
    def account_names(df):
        return np.unique(df['Account Name'])

    @staticmethod
    def theme_names(df):
        return np.unique(df['Theme Name'])

    @staticmethod
    def trader_names(df):
        return np.unique(df['Trader Name'])


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\troy\Anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\troy\Anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
today = date(2021, 9, 24)
objPortfolio = Portfolio(today)
trader_names = objPortfolio.trader_names(objPortfolio.raw_data)

data loaded ...


In [131]:
trader_name = 'troy'
removed_closed_trades = True

# Create the Pivot and round the numbers
df_trader = df[df["Trader Name"]=="troy"]
trades = pd.pivot_table(data=df_trader, values='#_or_notional_mio',
                        index=['Trade Name', 'Security Id'], columns='Account Name', margins=True, margins_name='Total', aggfunc='sum')
trade_df = trades.reset_index()
if 'MassPRIM-B' in trade_df.columns:
    trade_df = trade_df.drop(['MassPRIM-B'], axis=1)
trade_df = trade_df.round(2)

# Remove isins with 0 values
accounts = [v for v in np.unique(df_trader['Account Name']) if v!='MassPRIM-B']
zero_isins = (trade_df[accounts[0]]==0) | np.isnan(trade_df[accounts[0]])
for a in accounts:
    zero_isins = zero_isins & ((trade_df[a]==0) | (np.isnan(trade_df[a])))
trade_df = trade_df[~zero_isins]

# Remove duplicated Trade names for better display
trade_df['Trade Name'] = np.where(trade_df['Trade Name'].duplicated(), '', trade_df['Trade Name'])


def create_position_pivot(df):
    accounts = [v for v in np.unique(df['Account Name']) if v!='MassPRIM-B']
    
    # Create the Pivot
    trades = pd.pivot_table(data=df, values='#_or_notional_mio',
                        index=['Trade Name', 'Security Id'], columns='Account Name', margins=True, margins_name='Total', aggfunc='sum')
    trade_df = trades.reset_index()
    if 'MassPRIM-B' in trade_df.columns:
        trade_df = trade_df.drop(['MassPRIM-B'], axis=1)
    trade_df = trade_df.round(2)

    # Remove isins with 0 values
    zero_isins = (trade_df[accounts[0]]==0) | np.isnan(trade_df[accounts[0]])
    for a in accounts:
        zero_isins = zero_isins & ((trade_df[a]==0) | (np.isnan(trade_df[a])))
    trade_df = trade_df[~zero_isins]

    # Remove duplicated Trade names for better display
    trade_df['Trade Name'] = np.where(trade_df['Trade Name'].duplicated(), '', trade_df['Trade Name'])

    return trade_df

In [167]:
# Filter the option trades only
option_df = df_trader[(df_trader['Asset Type']=='Futures Options') | (df_trader['Asset Type']=='Swaption') | (df_trader['Asset Type']=='Currency Options')]
# Keep only open trades
option_df = option_df[option_df['Is Trade Open']=='TRUE']

# Split by asset Class
equity_option_df = option_df[option_df['Asset Class']=='Equity']
position_pivot = create_position_pivot(equity_option_df)

# Add a column to mark the expiry dates of the ISIN and thenindicate whether expired or alive
# df_trader[df_trader['Security Id'].isin(position_pivot['Security Id'])]
# position_pivot.join(df_trader[['Security Id', 'Maturity Date']], on='Security Id', how='left')
dts = []
for sid in position_pivot['Security Id']:
    filtered_df = df_trader[df['Security Id']==sid]
    if filtered_df.shape[0]>1:
        dts.append(filtered_df.iloc[0, :]['Maturity Date'])
    else:
        dts.append('')
position_pivot.insert(2, 'Maturity Data', dts)

C:\Users\troy\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [110]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

# Components
positions_table = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in trade_df.columns],
    data=trade_df.to_dict('records'),
    filter_action='native',
    style_cell={'textAlign': 'center', 'maxWidth': '100px'},
    style_header={
        'backgroundColor': 'gray',
        'fontWeight': 'bold', 
        'textAlign': 'center'
    },
    style_data={
        'whiteSpace': 'normal',
        'height': 'auto',
    },
    style_data_conditional=[
        {
            'if': {
                'filter_query': '{Trade Name} != ""',
            },
            'borderTop': '2px rgb(100, 100, 100) solid',
        },
        {
            'if': {
                'filter_query': '{MCMM} < 0',
                'column_id': 'MCMM'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{MassPRIM} < 0',
                'column_id': 'MassPRIM'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{PSAMF} < 0',
                'column_id': 'PSAMF'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{RVMaster} < 0',
                'column_id': 'RVMaster'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{UCITS} < 0',
                'column_id': 'UCITS'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {
                'filter_query': '{Total} < 0',
                'column_id': 'Total'
            },
            'color': 'tomato',
            'fontWeight': 'bold'
        },
        {
            'if': {'column_id': 'Trade Name'},
            'textAlign': 'left',
            'maxWidth': '150px'
        },
        {
            'if': {'column_id': 'Security Id'},
            'textAlign': 'left',
            'maxWidth': '150px'
        }
    ]
    )

table_filters = dcc.Checklist(
    options=[
        {'label': 'Remove Closed Trades', 'value': 'NoClosed'},
        {'label': 'Remove Matured ISINS', 'value': 'NoExpired'},
    ],
    value=['NoClosed', 'NoExpired'],
    labelStyle={'display': 'inline-block'}
)

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Portfolio Dashboard (Data: %s)" % today.strftime('%d-%m-%Y')),
    table_filters,
    positions_table
])

# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
